In [1]:
import pandas as pd
import geopandas as gpd
import shapely
from shapely.geometry import Point, LineString, MultiLineString, Polygon, MultiPolygon
from tqdm.notebook import tqdm
import time
from datetime import datetime

In [ ]:
# https://ipython-books.github.io/147-creating-a-route-planner-for-a-road-network/

In [2]:
gdf_big = gpd.read_file('./total/gdf_line_all_n_samobl.json',
                        encoding='utf-8')

In [3]:
gdf_left = gpd.read_file('./total/gdf_line_sam_tol_new2.json',
                        encoding='utf-8')

In [4]:
gdf_all = gdf_big.append(gdf_left)
gdf_all = gdf_all.drop_duplicates(subset=['line_id', 
                                          'trip_id']).reset_index(drop=True)

In [5]:
stps_big = pd.read_csv('./total/df_stop_seq_all_n_samobl.csv',
                        encoding='utf-8', sep=';')

In [6]:
graph = gpd.read_file('./with_ped/new_graph_with_ped.json',
                        encoding='utf-8')

In [7]:
nodes = gpd.read_file('./with_ped/all_nodes_with_ped.json',
                        encoding='utf-8')

In [8]:
one_big_line = gdf_all.iloc[[0]]


In [ ]:

# bg_geo = one_big_line.geometry[0]
# tripid = one_big_line.trip_id[0]

# lst_parts=[]
# i=0
# for i in range(len(bg_geo)):
#     one_part = bg_geo[i]
#     .append(one_part)
# # 

In [9]:
lst_parts = list(one_big_line.geometry[0])

In [10]:
lst_parts = list(one_big_line.geometry[0])

gdf_parts = gpd.GeoDataFrame(geometry=lst_parts)
gdf_parts.crs='epsg:4326'
gdf_parts['trip_id'] = one_big_line.trip_id[0]

In [12]:
tp_grph = graph[(((graph.type_ts == 'TM,CAR,BUS,TB,MT') 
                  | (graph.type_ts == 'TM')) 
                 & (graph.type_ped == 'car_n_ped') 
                 & (graph.lanes != 0))].reset_index(drop=True)

In [17]:
lst_frm = list(set(list(tp_grph.from_node.unique()) + list(tp_grph.to_node.unique())))

In [23]:
tp_nds = nodes[nodes.nodeID.isin(lst_frm)]

In [11]:
buff_gdf_parts = gdf_parts.copy()

buffer = 5
buff_gdf_parts.geometry = buff_gdf_parts.geometry.to_crs('epsg:32637').buffer(buffer).to_crs('epsg:4326')

In [13]:
%time grph_tmp = gpd.sjoin(tp_grph, buff_gdf_parts[['geometry']], how='inner', op='intersects')

Wall time: 269 ms


In [24]:
%time nds_tmp = gpd.sjoin(tp_nds, buff_gdf_parts[['geometry']], how='inner', op='intersects')

Wall time: 104 ms


In [31]:
def create_buff_frm_coord(coords,buffer):
    buff_gdf = gpd.GeoDataFrame(geometry=[Point(coords)])
    buff_gdf.crs='epsg:4326'
    buff_gdf.geometry = buff_gdf.geometry.to_crs('epsg:32637').buffer(buffer).to_crs('epsg:4326')
    return buff_gdf

In [29]:
i=0
one_line = lst_parts[i]
lst_coord = list(one_line.coords)
first = lst_coord[0]
buffer = 5

In [32]:
buff_gdf = create_buff_frm_coord(first,buffer)

In [38]:
last_buff_gdf = create_buff_frm_coord(lst_coord[-1],buffer)

In [36]:
first_sj = gpd.sjoin(tp_grph, buff_gdf[['geometry']], how='inner',
                       op='intersects').reset_index(drop=True)

In [39]:
last_sj = gpd.sjoin(tp_grph, last_buff_gdf[['geometry']], how='inner',
                       op='intersects').reset_index(drop=True)

In [45]:
buffer=5
cnt=1
bg_sj = gpd.GeoDataFrame()
# lst_parts = list(one_big_line.geometry[0])

i=0
for i in tqdm(range(len(lst_parts))):
#     one_buff = buff_gdf_parts.iloc[[i]].reset_index(drop=True)
    one_line = lst_parts[i]
    lst_coord = list(one_line.coords)
    first = lst_coord[0]
    last = lst_coord[-1]
    first_buff_gdf = create_buff_frm_coord(first,buffer)
    last_buff_gdf = create_buff_frm_coord(last,buffer)
    
    first_sj = gpd.sjoin(tp_grph, first_buff_gdf[['geometry']], how='inner',
                       op='within').reset_index(drop=True)
    if len(one_sj) > 0:
        one_sj['seq'] = cnt
        bg_sj = bg_sj.append(one_sj)
        cnt+=1
# 

In [48]:
bg_sj = bg_sj.reset_index(drop=True)
bg_sj = bg_sj.drop('index_right', axis=1)

In [53]:
bg_sj['trip_id']=buff_gdf_parts.trip_id[0]

In [46]:
one_stop_seq = stps_big[stps_big.trip_id == int(one_big_line.trip_id[0])].reset_index(drop=True)

In [132]:
buff_one_line = one_big_line.copy()

buffer = 7
buff_one_line.geometry = buff_one_line.geometry.to_crs('epsg:32637').buffer(buffer).to_crs('epsg:4326')

In [133]:
one_grph = gpd.sjoin(tp_grph, buff_one_line[['geometry','trip_id']],how='inner',op='within')

In [ ]:
# lst_parts = list(one_big_line.geometry[0])

# gdf_parts = gpd.GeoDataFrame(geometry=lst_parts)
# gdf_parts.crs='epsg:4326'
# gdf_parts['trip_id'] = one_big_line.trip_id[0]

In [53]:
ope_pt = gpd.GeoDataFrame(geometry=[Point(one_stop_seq.lon[0],one_stop_seq.lat[0])])

In [49]:
import momepy as mm
import networkx as nx

In [80]:
def get_closest_pt(ope_pt, one_grph, nodes):
    closest_link_id = mm.get_network_id(ope_pt, one_grph, 'link_id')[0]
    link_in_graph = one_grph[one_grph.link_id == closest_link_id].reset_index(drop=True)
    to_node_id = link_in_graph.to_node[0]
    from_node_id = link_in_graph.from_node[0]
    to_node = nodes[nodes.nodeID == to_node_id].reset_index(drop=True)['geometry'][0]
    from_node = nodes[nodes.nodeID == from_node_id].reset_index(drop=True)['geometry'][0]

    dist1 = ope_pt.geometry[0].distance(to_node)
    dist2 = ope_pt.geometry[0].distance(from_node)

    if dist1 < dist2:
        closest_node = to_node
        closest_node_id = to_node_id
    else:
        closest_node = from_node
        closest_node_id = from_node_id
    # 
    return closest_node,closest_node_id

In [81]:
%time closest_node,closest_node_id = get_closest_pt(ope_pt, one_grph, nodes)

Snapping: 100%|██████████████████████████████████████████████████████████████████████████| 1/1 [00:00<00:00, 71.44it/s]

Generating centroids...
Generating rtree...
Wall time: 68.2 ms


In [82]:
ope_pt2 = gpd.GeoDataFrame(geometry=[Point(one_stop_seq.lon[1],one_stop_seq.lat[1])])

In [83]:
%time closest_node2,closest_node_id2 = get_closest_pt(ope_pt2, one_grph, nodes)

Snapping: 100%|██████████████████████████████████████████████████████████████████████████| 1/1 [00:00<00:00, 76.92it/s]

Generating centroids...
Generating rtree...
Wall time: 65 ms


In [134]:
one_G = mm.gdf_to_nx(one_grph)

C:\Users\ochristm\anaconda3\lib\site-packages\momepy\utils.py:202: UserWarning: Geometry is in a geographic CRS. Results from 'length' are likely incorrect. Use 'GeoSeries.to_crs()' to re-project geometries to a projected CRS before this operation.

  gdf_network[length] = gdf_network.geometry.length


In [135]:
res_lst = nx.shortest_path(one_G, closest_node.coords[0], closest_node2.coords[0])

In [136]:
res_lst

[(50.1883528, 53.1915989),
 (50.1881586, 53.1915936),
 (50.1880451, 53.1915905),
 (50.1878873, 53.1915863),
 (50.18737, 53.1915722),
 (50.1867769, 53.1915562),
 (50.1840486, 53.1914827)]

In [ ]:
# def get_nodeid_by_coords(lst_coords_line, lst_coords_all, lst_ndid):



# return dct_ids

In [137]:
lst_nodid_line=[]
lst_seq =[]

cnt=1
i=0
for i in range(len(res_lst)):
    one_coord = res_lst[i]
    j=0
    if one_coord in lst_coords:
        ind = lst_coords.index(one_coord)
        nodid = lst_ndid[ind]
    else:
        print(i)
        nodid = 0
    #
    lst_nodid_line.append(nodid)
    lst_seq.append(cnt)
    cnt+=1
# 


In [138]:
lst_nodid_line

[27397, 27415, 27385, 27413, 26931, 27396, 26908]

In [131]:
lst_seq

[1, 2, 3, 4, 5, 6, 7]

In [ ]:
lst_all

In [107]:
nodes.geometry[0].coords[:]

[(49.7993692, 52.299129)]

In [124]:
# def create_dict_from_df(nodes):
lst_geo = list(nodes.geometry)
lst_ndid = list(nodes.nodeID)

lst_coords=[]
i=0
for i in range(len(nodes)):
    lst_coords.append(lst_geo[i].coords[0])
# 
# return dct

In [125]:
lst_coords_all[1]

(50.0882993, 53.1828105)

In [127]:
res_lst[1] in lst_coords

True